In [1]:
import os
import pandas as pd
import numpy
import csv
import requests
from bs4 import BeautifulSoup
import re
## openpyxl-3.1.5

In [5]:
!pip install openpyxl

In [51]:
def scraper(url="https://data.nsw.gov.au/data/dataset/fuel-check"):
    download_links = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href and (href.endswith(".csv") or href.endswith(".xlsx")) and ("24." in href or "25." in href):
                download_links.append(href)
        if download_links:
            print("Download links found (2024 or 2025):")
            for link in download_links:
                print(link)
        else:
            print("Not found")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
    except Exception as e:
        print(f"Error: {e}")
    return download_links

def combine(download_links):
    dataframes = []
    for url in download_links:
        if url.endswith(".xlsx"):
            df = pd.read_excel(url, engine="openpyxl")
        elif url.endswith(".csv"):
            df = pd.read_csv(url)
        else:
            continue
        dataframes.append(df)
    return dataframes
    final_df = pd.concat(dataframes, ignore_index=True)
    return final_df

def save(dataframes, loc='./final.csv'):
  #pd.concat(dataframes, ignore_index=True).to_csv(index=False)
  dataframes.to_csv(loc, sep='\t', encoding='utf-8', index=False, header=True)

In [36]:
df = combine(scraper())

Download links found (2024 or 2025):
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/2d785043-38eb-4eeb-b992-89f1af8d91e1/download/fuelcheck_pricehistory_jan2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/404dcdf6-bbde-4ccf-869e-259bcc408ce5/download/fuelcheck_pricehistory_feb2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/8c461685-3ed2-47e0-b9e6-9737c58a21bf/download/fuelcheck_pricehistory_mar2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/56a4fdb5-0789-4a2f-a89f-18e0159e210f/download/fuelcheck_pricehistory_apr2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/f70eb8e4-7291-47c8-874c-daea7eda0df1/download/fuelcheck_pricehistory_may2024.xlsx
https://data.nsw.gov.au/data/dataset/a97a46fc-2bdd-4b90-ac7f-0cb1e8d7ac3b/resource/7c6e7325-95ff-409e-becf-ffb45ad5271b/download/fuelcheck

In [40]:
final_df = pd.concat(df, ignore_index=True)

In [50]:
save(final_df)

In [43]:
final_df.head()

,ServiceStationName,Address,Suburb,Postcode,Brand,FuelCode,PriceUpdatedDate,Price
0,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,U91,2024-01-01 00:44:05,168.7
1,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,P98,2024-01-01 00:44:05,189.7
2,Costco Canberra Airport (Members Only),"39-41 Mustang Avenue, Canberra Airport ACT 2609",Canberra Airport,2609,Costco,PDL,2024-01-01 00:44:05,177.7
3,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,E10,2024-01-01 00:44:11,164.7
4,Costco Casula (Members only),"20 Parkersfarm Place, Casula NSW 2170",Casula,2170,Costco,P98,2024-01-01 00:44:11,188.7
